In [4]:
import pandas as pd

In [3]:
def bootstrap_validation(X_test, y_test, model, sample=100, random_state=88):

    n_sample = sample
    output_dict = {}

    output_dict['Accuracy'] = []
    output_dict['TPR'] = []
    output_dict['FPR'] = []
    output_dict['PRE'] = []
        
    for bs_iter in range(n_sample):
        bs_index = np.random.choice(X_test.index, len(X_test.index), replace=True)
        
        bs_data = X_test.loc[bs_index]
        bs_label = y_test.loc[bs_index]

        model_pred = model.predict(bs_data)
        cm = confusion_matrix(bs_label, model_pred)
        tn, fp, fn, tp = cm.ravel()
        output_dict['Accuracy'].append((cm.ravel()[0]+cm.ravel()[3])/sum(cm.ravel()))
        output_dict['TPR'].append(tp / (tp + fn))
        output_dict['FPR'].append(fp / (fp + tn))
        if tp+fp > 0:
            output_dict['PRE'].append(tp / (tp + fp))
        else:
            output_dict['PRE'].append(0)
            
    return output_dict

In [5]:
# bs_output = bootstrap_validation(X_test, y_test, logreg)

In [ ]:
CI_acc = np.quantile(bs_output['Accuracy'],np.array([0.025,0.975]))
CI_TPR = np.quantile(bs_output['TPR'],np.array([0.025,0.975]))
CI_FPR = np.quantile(bs_output['FPR'],np.array([0.025,0.975]))
CI_PRE = np.quantile(bs_output['PRE'],np.array([0.025,0.975]))

fig, axs = plt.subplots(ncols=4, figsize=(24,12))
i = 0
for metric, metric_results in bs_output.items():
    axs[i].hist(metric_results, label = metric)
    axs[i].set_xlabel(metric,fontsize=16)
    axs[i].set_ylabel('Count',fontsize=16)
    i+=1
    
plt.show()

In [ ]:
# Let's make a table for this 
bootstrap_data = {'Accuracy':[CI_acc[0],CI_acc[1]],'TPR':[CI_TPR[0],CI_TPR[1]],
                 'FPR':[CI_FPR[0],CI_FPR[1]],'PRE':[CI_PRE[0],CI_PRE[1]]}
bootstrap_table = pd.DataFrame(data = bootstrap_data, index = ['0.025 quantile','0.975 quantile']).transpose()
bootstrap_table.style.set_properties(**{'font-size': '12pt',}).set_table_styles([{'selector': 'th', 'props': [('font-size', '10pt')]}])
bootstrap_table